In [ ]:
#!/opt/miniconda3/bin/python
import pygmt
import json	# for reading bird plate-boundary data
import numpy as np
import os
import re
import sys
from os.path import exists as file_exists
from functools import reduce
#import slab2bird as sb   # loads slab2bird dictionary

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))


#### Read a reconstructed_0.00Ma.xy file

The format of a section of trench data looks like:

    1444 >SubductionZone <identity>GPlates-15619cbf-027b-4b8d-b6c2-82a7c05aba2d</identity> <revision>GPlates-0fdc33f6-816b-40d7-9e18-f3207544be40</revision>
    1445 > name Andean Trench Segment
    1446 > validTime TimePeriod <begin> TimeInstant <timePosition>11</timePosition></begin> <end> TimeInstant <timePosition>0</timePosition></end>
    1447 > reconstructionPlateId 201124
    1448 > subductionPolarity ConstantValue <value>Right</value> <valueType>SubductionPolarityEnumeration</valueType>
    1449 > subductionZoneDepth 80
    1450 > isActive ConstantValue <value> 1</value> <valueType>boolean</valueType>
    1451 > slabEdgeType Trench
    1452   -73.570086      -33.935762
    1453   -73.336070      -33.199409
    1454   -72.407817      -29.608626
    1455   -71.554108      -26.008878
    1456   -71.510612      -25.030648

##### the "reduce" function

    The reduce(fun,seq) function is used to apply a particular function passed in its argument to all of the list elements mentioned in the sequence passed along.This function is defined in “functools” module.

    Working :  

    At first step, first two elements of sequence are picked and the result is obtained.
    Next step is to apply the same function to the previously attained result and the number just succeeding the second element and the result is again stored.
    This process continues till no more elements are left in the container.
    The final returned result is returned and printed on console.

Note on how this works here:

    The "substrings_to_remove" is the list elements that are iteratively reduced to the second element of the function

    The "input_string" is taken as the frist element of the function

In [ ]:
in_file = os.path.join(ASPECT_LAB_DIR, "dtemp", "gplate_export_test0", "Muller_etal_2019_PlateBoundaries_no_topologies", "reconstructed_0.00Ma.xy")
assert(os.path.isfile(in_file))

# credit: chatgpt
# remove substrings in a string
remove_substrings = lambda input_string, substrings_to_remove: \
    reduce(lambda string, sub: re.sub(sub, '', string), substrings_to_remove, input_string)

# Read a file of plate reconstruction.
# We'll look for the key word "SubductionZone" in the file
# and save there respectvie coordinates into a big array
i=0
temp_l = []
temp_d = []
trench_data = []  # record the trench coordinates
trench_names = []
trench_pids = []
trench_begin_times = []
trench_end_times = []
n_trench = 0  # record the number of trenches
sbd_begin = False  # tags for in a section of trench data
sbd_end = False
read=True # a tag for continuing reading the file
with open(in_file, 'r') as fin:
    line = fin.readline()
    i += 1
    while line:
        read = True # by default, we will continue to read the file in each loop
        # First, in case we are reading a data section of trench locations,
        # check if we already reached the end of a section
        if sbd_begin and re.match('^>', line):
            sbd_end = True
        # Then, determine what to do.
        # 1. we are reading a data section and want to continue
        # 2. we are reading a data section and want to end
        # 3. we are not reading a data section and is haunting for the next
        # section in the file.
        if sbd_begin and (not sbd_end):
            # begin read in subduction zone data
            # To convert string inputs to data:
            #   first, split the string by any number of whitespace characters
            #   then, convert the data to floats and all the additional whitespace
            #   shoule be removed
            temp_data = line.split()
            temp_data = [float(x) for x in temp_data]
            temp_d.append(temp_data)
            pass
        elif sbd_begin and sbd_end:
            # reach the end of a section
            trench_data.append(temp_d)
            sbd_begin = False  # reset the two tags
            sbd_end = False
            read = False
            pass
        elif re.match('^>SubductionZone', line):
            # these inputs at the begining of a line
            # indicate the start of a subduction zone
            # data section.
            temp_l.append(i)
            sbd_begin = True # set the tag for the begining of the dataset
            temp_d = []
            # after finding the start of the section,
            # continue to read through the headers
            # of this section
            while(line and re.match('^>', line)):
                line=fin.readline()
                i += 1
                if re.match('^> name', line):
                    # look for the name of the trench and append it to the list
                    trench_names.append(remove_substrings(line, ["> name ", '\n']))
                elif re.match('> reconstructionPlateId', line):
                    # look for the plate id of the trench and append it to the list
                    trench_pids.append(remove_substrings(line, ["> reconstructionPlateId ", '\n']))
                elif re.match('> validTime TimePeriod <begin> TimeInstant <timePosition>', line):
                    # look for the begin and end time of the trench and append it to the list
                    temp0 = remove_substrings(line, ["> validTime TimePeriod <begin> TimeInstant <timePosition>", '</timePosition>.*\n'])
                    trench_begin_times.append(float(temp0))
                    temp1 = remove_substrings(line, ['^.*<end> TimeInstant <timePosition>', '</timePosition>.*\n'])
                    if type(temp1) == float:
                        trench_end_times.append(temp1)
                    else:
                        trench_end_times.append(0.0)
            read = False
        if read:
            # if the flag marks continue reading the file, proceed
            line = fin.readline()
            i += 1
i -= 1 # the last time reading is unsuccessful
n_trench = len(trench_data)



In [ ]:
# Set Regional Map dimensions
west = -280.0 # -143.0
east = -60 # -100.25
south = -50.0 # 32.0
north = 70
region_data = np.array([west, east, south, north])

dlon_edge = 0.35
dlat_edge = 1.0
region1 = str(west) + '/' + str(east) + '/' + str(south) + '/' + str(north) 
print('Region: ', region1)

# Get center position and two latitude locations to define regional projection
# this step is needed that both the centre values and the lat1, lat2 values need to be integars
# also by mutliplying 0.5, we are getting only half the length in the latitude dimension
clon = np.floor(region_data[0:2].mean())
clat = np.floor(region_data[2:4].mean())
lat1 = np.floor(0.5*(north - clat) + clat )
lat2 = np.floor(clat - 0.5*(clat - south) )

mapwidth = 4.5 # inches, sets size of figure on page
proj1 = 'B' + str(clon) + '/' + str(clat) + '/' + str(lat1) + '/' + str(lat2) + '/' + str(mapwidth) + 'i'
print('proj1: ', proj1)

# Start figure
# the pygmt.datasets contains data to be imported directly (e.g. earth_relief, earth_age)
# resolution: 30s (qs magali)
# order of the plot: first create the basemap (fig.basemap), then specific types of plots (e.g. color map - fig.grdimage)
fig = pygmt.Figure()
fig.basemap(region=region1, projection=proj1, frame=["a5f2.5", f'WSne'])
print('Plotting topo grid')
topogrid = pygmt.datasets.load_earth_relief(region=region1,resolution="02m",registration="pixel")
fig.grdimage(grid=topogrid,cmap="gray")
fig.show()

In [ ]:
# BUG: upon switching the color to red, the line segments are filled by color masses
for i_trench in range(n_trench):
    coordinates = trench_data[i_trench]
    fig.plot(coordinates, pen='2p,red')

fig.show()